## 문서 제작
1. DB.ipynb에서 통신사별 csv 추출
2. 경로 변경 후실행

In [1]:
pip install pandas python-docx

Note: you may need to restart the kernel to use updated packages.


### 통신사 docx 파일 만들기

In [8]:
import pandas as pd
from docx import Document
import os

def format_data_limit(val, charge_pln):
    """데이터 제공량 단위를 MB에서 GB로 변환하고 무제한 여부를 체크합니다."""
    if val >= 999999:
        return "무제한"
    if pd.isna(val):
        return "정보 없음"
    
    # MB -> GB 변환
    if val >= 1024:
        gb = val / 1024
        size_str = f"{int(gb)}GB" if gb == int(gb) else f"{gb:.1f}GB"
    else:
        size_str = f"{int(val)}MB"
    
    # 속도제어 등 추가 정보가 있다면 괄호 안에 표기
    if pd.notna(charge_pln) and charge_pln != "해당없음":
        return f"{size_str} ({charge_pln})"
    return size_str

def format_voice(val, charge_pln):
    """음성 제공량을 정리합니다."""
    if val >= 999999 or charge_pln == '무제한':
        return "제한없음"
    if pd.isna(val):
        return "정보 없음"
    return f"{int(val)}분"

def format_ott(nm, prd, mthd):
    """OTT 혜택 정보를 하나의 문자열로 합칩니다."""
    if pd.isna(nm):
        return "정보 없음"
    parts = [str(nm)]
    if pd.notna(prd): parts.append(str(prd))
    if pd.notna(mthd): parts.append(str(mthd))
    return " ".join(parts)

def parse_description(desc):
    """AI 설명에서 주요 특징과 추천 대상을 분리합니다."""
    if pd.isna(desc):
        return "정보 없음", "정보 없음"
    
    lines = desc.split('\n')
    # 첫 줄을 주요 특징으로 간주
    feature = lines[0].strip() if len(lines) > 0 else "정보 없음"
    
    # '추천대상:' 키워드가 포함된 라인 찾기
    target = "정보 없음"
    for line in lines:
        if "추천대상:" in line:
            target = line.replace("추천대상:", "").strip()
            break
    
    return feature, target

def convert_csv_to_word(csv_path, output_path):
    # 1. 데이터 로드
    df = pd.read_csv(csv_path)
    
    # 2. 워드 문서 객체 생성
    doc = Document()
    
    for index, row in df.iterrows():
        # 데이터 전처리
        name = row['PRC_PLN_NM']
        comp = row['COMP']
        price = f"{row['BASE_PRICE']:,}원/월"
        ott = format_ott(row['OTT_COMB_NM'], row['OTT_COMB_PRD'], row['OTT_COMB_MTHD'])
        feature, target = parse_description(row['AI_SHORT_DESC'])
        network = row['NW_TYPE']
        data_prov = format_data_limit(row['DATA_PRVD'], row['DATA_CHARG_PLN'])
        voice_prov = format_voice(row['VOICE_PRVD'], row['VOICE_CHARG_PLN'])
        
        # 3. 워드에 내용 추가
        doc.add_paragraph(f"# 요금제명 :[{comp}] {name}")
        doc.add_paragraph(f"통신사 : {comp}")
        doc.add_paragraph(f"이용료 : {price}")
        doc.add_paragraph(f"OTT 혜택 : {ott}")
        doc.add_paragraph(f"주요 특징 : {feature}")
        doc.add_paragraph(f"통신망 : {network}")
        doc.add_paragraph(f"데이터 제공량 : {data_prov}")
        doc.add_paragraph(f"음성 제공량 : {voice_prov}")
        doc.add_paragraph(f"추천 대상 : {target}")
        
        # 구분선 추가 (마지막 항목 제외)
        if index < len(df) - 1:
            doc.add_paragraph("-" * 3)
            
    # 4. 파일 저장
    doc.save(output_path)
    print(f"✅ 변환 완료: {output_path}")

# 실행
if __name__ == "__main__":
    input_csv = r'C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\data\lgu+.csv'  # 파일 경로
    output_word = r'C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\원본\LGU+_v2.docx'
    convert_csv_to_word(input_csv, output_word)

✅ 변환 완료: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\원본\LGU+_v2.docx


In [1]:
print("2")

2


## MD파일 만들기

In [12]:
import pandas as pd
import os

# --- 설정부 ---
# CSV 파일 경로
INPUT_CSV = r'C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\data\KT.csv' 
# 결과 마크다운 저장 경로
OUTPUT_MD = r'C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\md\KT_v2.md'

def format_data_limit(val, charge_pln):
    if val >= 999999: return "무제한"
    if pd.isna(val): return "정보 없음"
    if val >= 1024:
        gb = val / 1024
        size_str = f"{int(gb)}GB" if gb == int(gb) else f"{gb:.1f}GB"
    else:
        size_str = f"{int(val)}MB"
    if pd.notna(charge_pln) and charge_pln != "해당없음":
        return f"{size_str} ({charge_pln})"
    return size_str

def format_voice(val, charge_pln):
    if val >= 999999 or charge_pln == '무제한': return "제한없음"
    if pd.isna(val): return "정보 없음"
    return f"{int(val)}분"

def format_ott(nm, prd, mthd):
    if pd.isna(nm): return "정보 없음"
    parts = [str(nm)]
    if pd.notna(prd): parts.append(str(prd))
    if pd.notna(mthd): parts.append(str(mthd))
    return " ".join(parts)

def parse_description(desc):
    if pd.isna(desc): return "정보 없음", "정보 없음"
    lines = desc.split('\n')
    feature = lines[0].strip() if len(lines) > 0 else "정보 없음"
    target = "정보 없음"
    for line in lines:
        if "추천대상:" in line:
            target = line.replace("추천대상:", "").strip()
            break
    return feature, target

def convert_csv_to_md_v2(csv_path, output_path):
    if not os.path.exists(csv_path):
        print(f"❌ 파일을 찾을 수 없습니다: {csv_path}")
        return

    df = pd.read_csv(csv_path)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 각 요금제(엔티티)를 담을 리스트
    entity_list = []

    print(f"🚀 중복 구분자 제거 및 포맷 최적화 변환 시작... (총 {len(df)}건)")

    for index, row in df.iterrows():
        # 데이터 전처리
        name = row['PRC_PLN_NM']
        comp = row['COMP']
        price = f"{row['BASE_PRICE']:,}원/월"
        ott = format_ott(row['OTT_COMB_NM'], row['OTT_COMB_PRD'], row['OTT_COMB_MTHD'])
        feature, target = parse_description(row['AI_SHORT_DESC'])
        network = row['NW_TYPE']
        data_prov = format_data_limit(row['DATA_PRVD'], row['DATA_CHARG_PLN'])
        voice_prov = format_voice(row['VOICE_PRVD'], row['VOICE_CHARG_PLN'])
        
        # 1. 하나의 엔티티 문자열 생성 (보여주신 예시와 동일한 형식)
        # 헤더를 '# 요금제명 : 상품명' 형태로 구성
        entity_text = (
            f"# 요금제명 :[{comp}] {name}\n"
            f"통신사 : {comp}\n"
            f"이용료 : {price}\n"
            f"OTT 혜택 : {ott}\n"
            f"주요 특징 : {feature}\n"
            f"통신망 : {network}\n"
            f"데이터 제공량 : {data_prov}\n"
            f"음성 제공량 : {voice_prov}\n"
            f"추천 대상 : {target}"
        )
        entity_list.append(entity_text)
            
    # 2. 엔티티 리스트를 '---' 구분자 하나로 연결
    final_md_content = "\n\n---\n\n".join(entity_list)

    # 3. 파일 저장
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(final_md_content)
        
    print(f"✅ 변환 완료! 중복 구분선이 제거되었습니다: {output_path}")

if __name__ == "__main__":
    convert_csv_to_md_v2(INPUT_CSV, OUTPUT_MD)

🚀 중복 구분자 제거 및 포맷 최적화 변환 시작... (총 116건)
✅ 변환 완료! 중복 구분선이 제거되었습니다: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\통신사\md\KT_v2.md


### 카드은행

### 구독상품